In [1]:
import pymongo
with open('mongouri.txt', 'r') as file:
    mongo_uri = file.read()
type(mongo_uri)

str

In [2]:
import pymongo
from pymongo import MongoClient
from pymongo.server_api import ServerApi
import re
from search_functions import SearchInMongoDB

In [3]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
uri = mongo_uri

# Create a new client and connect to the server
client = MongoClient(uri)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


SSL handshake failed: ac-hnqzezj-shard-00-01.4hvj9hx.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1007) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: ac-hnqzezj-shard-00-00.4hvj9hx.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1007) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),SSL handshake failed: ac-hnqzezj-shard-00-02.4hvj9hx.mongodb.net:27017: [SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1007) (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 662003bd9c03932b30442b1e, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-hnqzezj-shard-00-00.4hvj9hx.mongodb.net', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('SSL handshake failed: ac-hnqzezj-shard-00-00.4hvj9hx.mongodb.

In [ ]:
app = SearchInMongoDB(uri, "users")

In [8]:
def display_results(results):
    if not results:
        print("No results found.")
        return
    print("\nResults:")
    for result in results:
        print(result)

## Searching Toy

In [ ]:
def interactive_search(app):
    while True:
        print("\nSelect a search option:")
        print("1. Search by Name or Screen Name")
        print("2. Search by ID")
        print("3. Search by Favourites Count")
        print("4. Search by Location")
        print("5. Search by Followers Count")
        print("6. Exit")
        choice = input("Choice: ")

        if choice == "1":
            name = input("Enter the name or screen name to search: ")
            search_type = input("Enter 'fuzzy' for fuzzy search or 'exact' for exact search: ")
            results = app.search_by_name(name, search_type)
            display_results(results)
        elif choice == "2":
            id_str = input("Enter the user ID to search: ")
            results = app.search_by_id_str(id_str)
            display_results(results)
        elif choice == "3":
            min_favourites = int(input("Enter minimum favourites count: "))
            max_favourites = input("Enter maximum favourites count (or leave blank): ")
            max_favourites = int(max_favourites) if max_favourites else None
            results = app.search_by_favourites(min_favourites, max_favourites)
            display_results(results)
        elif choice == "4":
            location = input("Enter the location to search: ")
            results = app.search_by_location(location)
            display_results(results)
        elif choice == "5":
            min_followers = int(input("Enter minimum number of followers: "))
            results = app.search_by_followers_count(min_followers)
            display_results(results)
        elif choice == "6":
            print("Exiting search...")
            break
        else:
            print("Invalid choice. Please enter a valid option.")



Divya

Alternate in notebook

In [ ]:
class SearchInMongoDB(object):
    def __init__(self, uri, collection_name, search_string):
        self.client = MongoClient(uri)
        self.db = self.client["twitter_database"]
        self.collection = self.db[collection_name]

    def interactive_search(self):
        while True:
            print("\nSelect a search option:")
            print("1. Search by Name or Screen Name")
            print("2. Search by ID")
            print("3. Search by Favourites Count")
            print("4. Search by Location")
            print("5. Search by Followers Count")
            print("6. Exit")
            choice = input("Choice: ")

            if choice == "1":
                search_string = input("Enter name or screen name to search: ")
                results = self.fuzzy_search_by_name(search_string)
                self.display_results(results)
            elif choice == "2":
                id_str = input("Enter user ID to search: ")
                results = self.search_by_id_str(id_str)
                self.display_results(results)
            elif choice == "3":
                self.search_by_favourites()
            elif choice == "4":
                location = input("Enter location to search: ")
                results = self.search_by_location(location)
                self.display_results(results)
            elif choice == "5":
                min_followers = int(input("Enter minimum number of followers: "))
                results = self.search_by_min_followers(min_followers)
                self.display_results(results)
            elif choice == "6":
                print("Exiting search...")
                break
            else:
                print("Invalid choice. Please enter a valid option.")

    def fuzzy_search_by_name(self, search_string):
        query = {"$or": [
            {"name": {"$regex": re.escape(search_string), "$options": "i"}},
            {"screen_name": {"$regex": re.escape(search_string), "$options": "i"}}
        ]}
        return list(self.collection.find(query))

    def search_by_id_str(self, id_str):
        return list(self.collection.find({"id_str": id_str}))

    def search_by_favourites(self):
        min_favourites = int(input("Enter minimum favourites count: "))
        results = self.collection.find({"favourites_count": {"$gte": min_favourites}}).sort("favourites_count", -1)
        self.display_results(results)

    def search_by_location(self, location):
        query = {"location": {"$regex": re.escape(location), "$options": "i"}}
        return list(self.collection.find(query))

    def search_by_min_followers(self, min_followers):
        return list(self.collection.find({"followers_count": {"$gte": min_followers}}))

    def display_results(self, results):
        if not results:
            print("No results found.")
            return
        print("\nResults:")
        for result in results:
            print(result)


## Driver


In [ ]:
def get_search_string():
    print("=== Welcome to MongoDB Search ===")
    print("Enter the search string to find users:")
    search_string = input("Search: ")
    print("================================")
    return search_string

def display_search_results(results, search_type):
    if results:
        print(f"=== Search Results for {search_type} ===")
        for idx, doc in enumerate(results, 1):
            print(f"Result {idx}:")
            print(doc)
            print("--------------------------------")
        print("================================")
    else:
        print(f"No results found for {search_type}.")

# Replace 'your_collection_name' with the actual collection name
collection_name = "users"

# Get the search string from the user
search_string = get_search_string()

# Create an instance of SearchInMongoDB
mongo_search = SearchInMongoDB(search_string, collection_name)

# Display fuzzy search results
display_search_results(mongo_search.fuzzy_search_result, "Fuzzy Search")

# Display exact search results
display_search_results(mongo_search.exact_search_result, "Exact Search")
    
# # Replace 'your_collection_name' with the actual collection name
# collection_name = "users"

# # Provide the search string
# search_string = "Uau"

# # Create an instance of SearchInMongoDB
# mongo_search = SearchInMongoDB(search_string, collection_name)

# print("HERE IS THE RESULT FOR FUZZY SEARCH")
# for doc in mongo_search.fuzzy_search_result:
#     print(doc)

# print("NEXT IS THE RESULT FOR EXACT SEARCH")
# for doc in mongo_search.exact_search_result:
#     print(doc)


# Testing Class